In [247]:
import numpy as np
import pandas as pd
import math


In [248]:
articles_df = pd.read_csv("./data/shared_articles.csv")
interactions_df = pd.read_csv("./data/users_interactions.csv")


In [249]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)


In [250]:
articles_df.query("eventType == 'CONTENT SHARED'").count()


timestamp          3047
eventType          3047
contentId          3047
authorPersonId     3047
authorSessionId    3047
authorUserAgent     669
authorRegion        669
authorCountry       669
contentType        3047
url                3047
title              3047
text               3047
lang               3047
dtype: int64

In [251]:
# Задание 6.2

# Создайте признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами).
# Вычислите среднее значение для полученного признака. Округлите его до двух знаков после точки-разделителя.

event_type = {
    'VIEW': 1.0,
    'LIKE': 2.0,
    'BOOKMARK': 2.5,
    'FOLLOW': 3.0,
    'COMMENT CREATED': 4.0,
}

interactions_df['event_code'] = interactions_df['eventType'].map(event_type)
interactions_df['event_code'].mean().round(2)


1.24

In [252]:
# Задание 6.3

# Чтобы получить хоть какую-то информацию, на которую можно будет опираться, оставьте только тех пользователей, которые
# взаимодействовали хотя бы с пятью статьями. Сколько всего таких пользователей?
interactions_counter = interactions_df.groupby(
    by=['personId', 'contentId']).first().reset_index().groupby(by=['personId']).size()

users_with_enough_interactions_df = \
    interactions_counter[interactions_counter >= 5].reset_index()

users_with_enough_interactions_df.rename(columns={0: "counter"}, inplace=True)

users_with_enough_interactions_df.shape[0]


1140

In [253]:
# Задание 6.4

# Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей (то есть тех, которые взаимодействовали
# как минимум с пятью статьями). Сколько всего таких взаимодействий?

mask = users_with_enough_interactions_df['personId']
selected_interactions_df = interactions_df[interactions_df['personId'].isin(
    mask)]
selected_interactions_df.shape[0]


69868

In [254]:
# Задание 6.5

# Примените упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной
# статьёй. Также сохраните для каждой пары «пользователь - статья» значение времени последнего взаимодействия.
# Найдите среднее по признаку с получившимися временными отсечками. Округлите результат до двух знаков после точки-разделителя.

def smooth_user_preference(x):
    return math.log(1+x, 2)


sum_interactions_log = selected_interactions_df.groupby(by=['personId', 'contentId'])[
    'event_code'].sum().apply(smooth_user_preference).reset_index()

sum_interactions_log = pd.DataFrame(sum_interactions_log).rename(
    columns={"event_code": "sum_attention"})

max_timestamp = selected_interactions_df.groupby(by=['personId', 'contentId'])[
    'timestamp'].max().reset_index()

sum_interactions_log = pd.merge(sum_interactions_log, max_timestamp, on=[
                                'personId', 'contentId'])
sum_interactions_log = sum_interactions_log.reindex(
    columns=['personId', 'contentId', 'timestamp', 'sum_attention'])

sum_interactions_log['timestamp'].mean().round(2)


1470605340.04

In [255]:
# Задание 6.6

# Разделите данные на обучающую и тестовую выборки. выбрав в качестве временной отсечки значение 1475519545. Значение отсечки включите
# в тестовую выборку. Сколько объектов попало в обучающую выборку?
#sum_interactions_log = sum_interactions_log.sort_values(by='timestamp')

time_gate = 1475519545

mask = sum_interactions_log['timestamp'] >= time_gate

train_df = sum_interactions_log.query("timestamp < @time_gate")
test_df = sum_interactions_log.query("timestamp >= @time_gate")

# train_df = train_df.rename(columns={'sum_attention': 'sum_attention_train'})
# test_df = test_df.rename(columns={'sum_attention': 'sum_attention_test'})

print(f'Train_shape: {train_df.shape[0]}, Test_shape: {test_df.shape[0]}')


Train_shape: 29325, Test_shape: 9781


In [256]:
# Для удобства дальнейшего измерения качества рекомендаций преобразуйте данные так, чтобы получить таблицу в формате, где строка
# соответствует пользователю, а столбцы будут истинными предпочтениями и рекомендациями в формате списков. На место пустых ячеек
# поместите пустые списки.

final_df = (
    train_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    test_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [
    [] if x is np.NaN else x for x in final_df['true_test']]
final_df.tail()


,true_train,true_test
personId,,
953707509720613429,"[-1068603220639552685, -2358756719610361882, -4503975842879662368, -6156751702010469220, -6783772548752091658, -8043889175123948305, -908052164352446106, 4105873627900556540, 5658521282502533116, 6943483335823457240, 7087600436874507849, 8074780316213343498, 8749720044741011597]","[-2402288292108892893, -5813211845057621660, -6444494220871855741, -6728844082024523434, 991271693336573226]"
983095443598229476,"[-133139342397538859, -8742648016180281673]","[-14569272361926584, -1572252285162838958, -1878128207048892154, -4333957157636611418, -6623581327558800021, -6728844082024523434, -7294716554902079523, 2103268612948910635, 2231749805468232267, 4891010769166501207, 5429045128339158843, 569574447134368517, 5847042211895226591, 8869347744613364434]"
989049974880576288,"[-133139342397538859, -2038869595290705317, -271938775771328391, -3959242148361340089, -5200964355640106406, -5380376324140474506, -7492067511846042603, -820343972901090172, -8742648016180281673, -8757701761088346244, 2267321579404324900, 2602738827388230683, 4277778562135834208, 7534917347133949300]","[-6289909056857931861, -7047448754687279385, -7215110246375999491, 6031953227014493100, 8526042588044002101]"
997469202936578234,"[-2358756719610361882, -4336877432539963613, -4655195825208522542, -5078536505040226358, -6603970730135147059, 2262656830196923781, 2602221055863892168, 4375556914674736641, 464601875539492971, 521382866757596943, 6910763794618680440, 9112765177685685246]","[-4029704725707465084, -5920475612630001479, -7047448754687279385, 2834072258350675251, 8869347744613364434]"
998688566268269815,"[-1255189867397298842, -401664538366009049, 6881796783400625893, 739747367187387064]","[3456674717452933449, 7174452660053929140]"


In [261]:
# Задание 6.7

# Посчитайте популярность каждой статьи как сумму всех логарифмических «оценок» взаимодействий с ней (используя только обучающую
# выборку). Выберите ID самой популярной статьи:

popular = train_df.groupby(by='contentId')['sum_attention'].sum(
).reset_index().sort_values('sum_attention',ascending=False)
popular.head()


,contentId,sum_attention
805,-6783772548752091658,231.177195
43,-133139342397538859,228.024567
1002,-8208801367848627943,189.937683
2211,8224860111193157980,186.044680
2111,7507067965574797372,179.094002


In [262]:
# Задание 6.8

# Постройте систему рекомендаций. Оцените качество с помощью precision@10 для каждого пользователя (доля угаданных рекомендаций). После
# этого усредните результат по всем пользователям.
# Для вычисления precision@10 воспользуйтесь следующей функцией:


In [258]:
most_popular = popular.head(10)
prediction = set(most_popular['contentId'].tolist())

def predict(sequence):
    sequence = set(sequence)
    return list(prediction - sequence)

final_df['train_pred'] = final_df['true_train'].apply(predict)

In [259]:
def precision(column):
    return (
        final_df
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)).mean()
precision('train_pred')

0.004285989399957339

---